In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import os

In [2]:
def to_df(tbl):  # tbl has type of bs4.element.Tag
    tb = []

    for r in tbl.find_all('tr'):
        tr = []
        for td in r.find_all('td'):
            txts = []
            for c in td.children:
                if c.text.strip():
                    txts.append(c.text.strip())
            txt = '\n'.join(txts)
            tr.append(txt)
        tb.append(tr)
    for i1, r in enumerate(tb):
        for i2, c in enumerate(r):
            if c == '':
                tb[i1][i2] = None
    return pd.DataFrame(tb)

In [3]:
def to_df_2(table):
    trs = table.find_all('tr')
    df = []
    for r in trs:
        td = r.find_all('td')
        cs = []
        for d in td:
            csi = []
            for c in d.children:
                if c.name == 'br':
                    pass
    #                 print(c.text.strip(), end=',')
                else:
                    if c.name == 'ol':
                        lis = c.find_all('li')
                        for li in lis:
                            csi.append(li.text.strip())
                    else:
                        if c.text.strip() != '':
                            csi.append(c.text.strip())
            cs.append(csi)
        df.append(pd.DataFrame(cs).T)

    return pd.concat(df, ignore_index=True)

In [4]:
def extract_tbs_all(f, start, end, extractor):
    res = []
    html = BeautifulSoup(open(f).read(), 'lxml')
    tables_bs = html.find_all('table')[start:end+1]
    for i, tbs in enumerate(tables_bs):
        print(i)
        df = to_df(tbs)
        res.append(extractor(df))
        
    return res

In [5]:
proc_dir = '../output/stage1/'
out_dir = '../output/stage2'

In [1197]:
fs = [x for x in os.listdir(proc_dir) if 'html' in x and '10' in x]

In [1199]:
fs

['Afghanistan 2010-11 MICS_English.html',
 'Bhutan 2010 MICS_English.html',
 'Central African Republic 2010 MICS_French.html',
 'Chad 2010 MICS_French.html',
 'Congo DR 2010 MICS_French.html',
 'Gambia 2010 MICS_English.html',
 'Ghana (Accra) 2010-11 MICS_English.html',
 'Guinea-Bissau 2010 MICS_Portuguese.html',
 'Kazakhstan 2010-11 MICS_English.html',
 'MICS4_Final_Togo_Fr_2010.html',
 'Mongolia 2010 MICS_English.html',
 'Myanmar 2009-10 MICS_English.html',
 'Nepal (Mid and Far Western Regions) 2010 MICS_English.html',
 'Pakistan (Balochistan) 2010 MICS_English.html',
 'Serbia (Roma Settlements) 2010 MICS_English.html',
 'Serbia (Roma Settlements) 2010 MICS_Serbian.html',
 'Serbia 2010 MICS_English.html',
 'Sierra Leone 2010 MICS_English.html',
 'South Sudan 2010 MICS_English.html',
 'State of Palestine 2010 MICS_English.html',
 'Sudan 2010 MICS Summary_English.html',
 'Suriname 2010 MICS_English.html',
 'Swaziland 2010 MICS_English.html']

In [1198]:
len(fs)

23

In [1200]:
i = 0

f = os.path.join(proc_dir, fs[i])
f_ = os.path.basename(f)

html = BeautifulSoup(open(f).read(), 'lxml')

In [1201]:
tables_bs = html.find_all('table')[:60]
f_

'Afghanistan 2010-11 MICS_English.html'

In [6]:
s1 = set([x[:-4] for x in os.listdir('../source/to_proceed/') if 'pdf' in x])

In [7]:
s2 = set([x[:-4] for x in os.listdir('../output/stage2/') if 'csv' in x])

In [8]:
d = list(s1.difference(s2))

In [9]:
len(d)

86

In [455]:
i = 86


f = os.path.join(proc_dir, d[i]+'.html')
f_ = os.path.basename(f)

html = BeautifulSoup(open(f).read(), 'lxml')

IndexError: list index out of range

In [456]:
tables_bs = html.find_all('table')[:60]
f_

'Belize 2015-16 MICS KFR_English.html'

In [1960]:
# def re_arrange_rows
# def fill_columns

In [448]:
to_df_2(tables_bs[47])

,0,1,2,3
0,Alcohol use,NaN,NaN,NaN
1,MICS,Indicator,Description,Value
2,Indicator,None,None,None
3,12.3,Use of alcohol,Percentage of people age 15-49 years who had a...,29.3
4,None,None,Women,53.7
5,None,None,Men,None
6,12.4,Use of alcohol before age 15,Percentage of people age 15-49 years who had a...,4.5
7,None,None,Women,18.7
8,None,None,Men,None


In [454]:
res = [to_df_2(tables_bs[x]) for x in range(7,48)]
res = pd.concat([x for x in res if len(x.columns) < 6) > 2], ignore_index=True)
res.to_csv(os.path.join(out_dir, f_.replace('.html', '.csv')), index=False)

In [599]:
def fill_col_2007(df, sh=1):
    for i, row in df.iterrows():
        if row.ix[0] and re.match('^[0-9]+$', row.ix[0]):
            df.ix[i] = row.shift(1)
        if len(row.dropna()) > sh and re.match('^[0-9\.]+', row.dropna().values[-1]):
            while not df.iloc[i, -1]:
                df.ix[i] = row.shift(1)
    return df

In [1134]:
def fill_col_2007_2(df, sh=2):
    df = df.dropna(how='all', axis=1).copy()
    
    for i, row in df.iterrows():
        if row.ix[0] and re.match('^[0-9]+$', row.ix[0]):
            df.ix[i] = row.shift(1)
        if len(row.dropna()) == sh and re.match('^[0-9\.]+', row.dropna().values[-2]) and len(row.dropna().values[-1]) < 20:
            while not df.iloc[i, -1]:
                df.ix[i] = row.shift(1)
                
    for i, row in df.iterrows():
#         if row.values[2]:
#             print(row.values[2])
        if not pd.isnull(row.values[2]) and not re.match('^[1-9]+[a-z]?$', row.values[2]):
            if len(row.dropna().values) == 2:
                while not df.iloc[i, -1]:
                    df.ix[i] = row.shift(1)
            else:
                while not df.iloc[i, -1]:
                    df.ix[i, 2:] = row[2:].shift(1)
    return df

In [1359]:
def push_2007_2(df, n=1, rex='^[0-9\.\n]+$', m=None):
    for i, row in df.iterrows():
#         if row.values[2]:
#             print(row.values[2])
        if not pd.isnull(row[n]) and re.match(rex, row[n]):
            mm = m
            while not df.ix[i].iloc[-1]:
                df.ix[i, n:] = row[n:].shift(1)
                if mm:
                    mm = mm - 1
                    if mm < 1:
                        break
    return df

In [572]:
fill_col_2007(to_df(tables_bs[1]))

,0,1,2,3,4,5
0,Rubrique,Numéro Indicateur MICS,Numéro Indicate ur\nOMD,Indicateur,None,Valeur
1,Sources et coûts des moustiquaires imprégnées ...,96\n97,None,Source d’approvisionnement (sources publiques)...,37,pour cent
2,NaN,NaN,NaN,NaN,39,pour cent
3,NaN,NaN,NaN,NaN,59,pour cent
4,NaN,NaN,NaN,NaN,69,pour cent
5,NaN,NaN,NaN,NaN,3500,francs CFA
6,NaN,NaN,NaN,NaN,5000,francs CFA
7,NaN,NaN,NaN,NaN,1800,francs CFA
8,NaN,NaN,NaN,NaN,1930,francs CFA
9,NaN,NaN,NaN,NaN,1600,francs CFA


In [1360]:
res = [fill_col_2007(to_df(tables_bs[x]), 3).drop(0) for x in range(0,7)]

res = pd.concat(res, ignore_index=True)

In [558]:
res.columns = ['topic', 'mics indicator', 'mdg indicator', 'name', 'value', 'unit']

In [1979]:
res = [to_df_2(tables_bs[x]) for x in range(3,5)]
res = pd.concat(res, ignore_index=True)

In [1980]:
# res = push_2007_2(res, 2, '^[A-Za-z]*', m=1)

In [1981]:
res.to_csv(os.path.join(out_dir, f_.replace('.html', '.csv')), index=False)

In [ ]:
# 2009

In [446]:
to_df(tables_bs[0])

,0,1,2,3,4,5,6
0,TopIC,MICS INDICATor\nNUMber,MDG\nlINkAGe,INDICATor,VAlUe IN 2000,VAlUe IN 2006,UNIT
1,CHIlD MorTAlITY,None,None,None,None,None,None
2,Child mortality,1,13,Under-five mortality rate,72.01,47.0,per\nthousand
3,2,14,Infant mortality rate,54.01,37.0,per\nthousand,None
4,NUTrITIoN,None,None,None,None,None,None
5,Nutritional status,6,4,Underweight prevalence,13.6,12.4,percent
6,7,None,Stunting prevalence,10.8,13.7,percent,None
7,8,None,Wasting prevalence,10.6,7.6,percent,None
8,Breastfeeding,45,None,Timely initiation of breastfeeding (within 1 h...,NA,43.1,percent
9,15,None,Exclusive breastfeeding rate (under 6 months),15.32,21.4,percent,None


In [404]:
def fill_col_2009(df):
    num_re = re.compile('^[0-9\.\,\n]+$')
    id_re = re.compile('^[1-9\.]+ .+$')
    
    for i, row in df.iterrows():
        r = row.dropna().values
        if len(r) == 2:
            if num_re.match(r[0]):
                while not df.iloc[i, -1]:
                    df.ix[i] = row.shift(1)
        elif len(r) == 3:
            if num_re.match(r[1]):
                if id_re.match(r[0]):
                    if not row.values[1] == r[0]:
                        df.iloc[i, 1] = r[0]
                else:
                    while not df.iloc[i, -1]:
                        df.ix[i] = row.shift(1)
        elif len(r) > 3:
            if num_re.match(r[-2]):
                if id_re.match(r[1]):
                    if not row.values[1] == r[1]:
                        df.iloc[i, 1] = r[1]
                else:
                    while not df.iloc[i, -1]:
                        df.ix[i] = row.shift(1)
        
                    
        matches = df.ix[:i, 1].str.match('[1-9]+')
        if len(matches[matches==True]) > 0:
            last_idx = matches[matches==True].index[-1]
            df.ix[i, 1] = '\n'.join(df.ix[last_idx:i, 1].dropna().values)
    
    return df

In [381]:
def process_row_2009(row):    
    r = row.dropna().values
    if len(r) < 3:
        return

    unit = r[-1]
    value = r[-2]
    
    num_re = re.compile('^[0-9\.\,]+')
    if not num_re.match(value):
        return
    try:
        indicator = r[-3].split(' ')[0]
        name = ' '.join(r[-3].split(' ')[1:])
    except:
        print(r)
    
    if '\n' in value:
        for v in value.split('\n'):
            if v.strip() != '':
                yield (indicator, name, v, unit)
    else:
        yield (indicator, name, value, unit)

In [369]:
def extract_2009(df):
    res = []
    df = fill_col_2009(df)
    for i, row in df.iterrows():
        for a, b, c, d in process_row_2009(row):
            res.append((a, b, c, d))
    return pd.DataFrame(res, columns=['id', 'name', 'value', 'unit'])

In [405]:
fill_col_2009(to_df(tables_bs[2]))

,0,1,2,3,4
0,Topic,MICS4 MDG\nIndicator Indicator Indicator Numbe...,Value & Unit,None,None
1,"HIV/AIDS, SEXUAL BEHAVIOUR, AND ORPHANED AND V...",None,None,None,None
2,HIV/AIDS knowledge and attitudes,9.1 Comprehensive knowled...,42.8,Percent,None
3,NaN,9.1 Comprehensive knowled...,women aged 15-24 years,42.1,Percent
4,NaN,9.1 Comprehensive knowled...,NaN,52.7,Percent
5,NaN,9.1 Comprehensive knowled...,NaN,39.0,Percent
6,NaN,9.1 Comprehensive knowled...,NaN,92.9,Percent
7,NaN,9.1 Comprehensive knowled...,NaN,69.2,Percent
8,NaN,9.1 Comprehensive knowled...,NaN,64.2,Percent
9,NaN,9.1 Comprehensive knowled...,NaN,78.6,Percent


In [413]:
res = extract_tbs_all(f, 0, 1, extract_2009)

0
1


In [414]:
(pd.concat(res, ignore_index=True)[['id', 'name', 'value', 'unit']]
 .to_csv(os.path.join(out_dir, f_.replace('.html', '.csv')), index=False))

In [10]:
# 2010

In [160]:
def fill_column_2010(df):
    num_re = re.compile('^([0-9\.\,]+)([^\-a-zA-Z]*)$')
    id_re = re.compile('^[1-9]+|^TA|^ТА|^SW|^MТ|^MT')
    
    for i, r in df.iterrows():
        if r[0] and id_re.match(r[0]):
            df.ix[i] = r.shift(1)
        if r[1] and num_re.match(r[1]):
            r[0] = None
            while not df.iloc[i, -1]:
                df.ix[i] = r.shift(1)
            
    # search last valid index
    
    df[1] = df[1].fillna(method='ffill')

    return df

In [283]:
def process_row_2010(row):
    num_re = re.compile('^([0-9\.\,]+)([^\-]*)')
    id_re = re.compile('[1-9\.]+|TA|ТА|SW|MT|МТ|CS')
    
    r = row.dropna().values
    
    if len(r) < 2:
        return
    if not num_re.match(r[-2]):
        return
#     print(r.values)

    indicator = None
    name = None
    value = r[-2]
    unit = r[-1]
    
    for i in range(len(r)-2):
#         print(r[i])
        if id_re.match(r[i]):
            indicator = r[i]
            break
    else:
        print(row)
        i = 0
    for j in range(i+1, len(r)-2):
        if not id_re.match(r[j]):
            name = r[j]
            break
    
    if '\n' not in value:
        value = num_re.match(r[-2]).groups()[0]
        if not unit:
            unit = num_re.match(r[-2]).groups()[1]
            if unit.strip() == '':
                unit = None
        yield (indicator, name, value, unit)
    else:
        
        ids = indicator.split('\n')
        values = value.split('\n')
        for i in range(len(values)-1, -1, -1):
            v, u = num_re.match(values[i]).groups()
            if u.strip() == '':
                u = None
            try:
                yield (ids[i], name, v, u)
            except:
                yield (indicator, name, v, u)

In [332]:
to_df(tables_bs[7])

,0,1,2,3
0,Etat Nutritionneli,None,None,None
1,Indicateur MICS,Indicateur,Description,Valeur
2,2.1a OMD 1.8 Prévalence de l’insuffisance Pour...,"12,3\n3,3",None,None
3,2.2a\n2.2b,Prévalence du retard de croissance\nModérée et...,Pourcentage d’enfants de moins de 5 ans qui se...,"21,2\n8,2"
4,2.3a\n2.3b,Prévalence de l’émaciation\nModérée et Sévère ...,Pourcentage d’enfants de moins de 5 ans qui se...,"8,2\n2,6"
5,2.3c,Prévalence de l’obésité,Pourcentage d’enfants de moins de 5 ans qui se...,"5,9"


In [138]:
fill_column_2010(to_df(tables_bs[4]))

,0,1,2,3,4,5
0,Topic,MICS4\nIndicator Number,MDG\nIndicator Number,Indicator,Value,None
1,None,9.12,None,Age-mixing among sexual partners Women age 15-...,"7,9\n2,5",percent percent
2,None,9.13,None,Sex with multiple partners\nWomen age 15-24\nM...,"1,2\n16,6",percent percent
3,None,9.14,None,Condom use during sex with multiple partners W...,"73,5\n76,2",percent percent
4,None,9.15,None,Sex with non-regular partners Women age 15-24\...,"7,4\n38,6",percent percent
5,None,9.16,6.2,Condom use with non-regular partners Women age...,"69,9\n78,3",percent percent
6,TOBACCO AND ALCOHOL USE,9.16,None,None,None,None
7,Tobacco Use,ТА.1,None,Tobacco use\nWomen age 15-49\nMen age 15-49,"7,5\n53,9",percent percent
8,ТА.2,ТА.1,Smoking until age 15\nWomen age 15-49\nMen age...,"1,3\n8,7",percent percent,None
9,Alcohol Use,ТА.3,None,Alcohol use\nWomen age 15-49\nMen age 15-49,"26,6\n46,4",percent percent


In [276]:
def extract_2010(t, fill_col=True, fill_na=False):
    
    res = []
    
    if fill_col:
        t = fill_column_2010(t)
        
    if fill_na:
        t[1] = t[1].fillna(method='ffill')
    
    for i, row in t.iterrows():
        for a, b, c, d in process_row_2010(row):
            res.append((a,b,c,d))

    return pd.DataFrame(res, columns=['id', 'name', 'value', 'unit'])

In [299]:
res = extract_tbs_all(f, 0, 3, extract_2010)

0
0                                                 None
1    Underweight prevalence\n2.1a 1.8 Moderate and ...
2                                                  5.8
3                                              Percent
Name: 2, dtype: object
0                                                  NaN
1    Underweight prevalence\n2.1a 1.8 Moderate and ...
2                                                  1.3
3                                              Percent
Name: 3, dtype: object
0                                                  NaN
1    Underweight prevalence\n2.1a 1.8 Moderate and ...
2                                             8.8\n2.2
3                                      Percent Percent
Name: 4, dtype: object


AttributeError: 'NoneType' object has no attribute 'split'

In [263]:
(pd.concat(res, ignore_index=True)[['id', 'name', 'value', 'unit']]
 .to_csv(os.path.join(out_dir, f_.replace('.html', '.csv')), index=False))

In [ ]:
# 2012 type 1. there may be 2 columns of values

In [53]:
# 2012, 2013 type 2

In [224]:
def process_row_2012(r):
    num_re = re.compile('^([0-9\.\,]+)([^\-]*)')
    id_re = re.compile('^[1-9]+|^TA|^SW|^MT')
    
    r = r.dropna().values
    
    if len(r) < 2:
        return
    if not num_re.match(r[-1]):
        return
#     print(r.values)

    indicator = None
    name = None
    value = r[-1]
    unit = None
    
    for i in range(len(r)-1):
        if id_re.match(r[i]):
            indicator = r[i]
            break
    for j in range(i+1, len(r)-1):
        if not id_re.match(r[j]):
            name = r[j]
            break
    
    if '\n' not in value:
        value, unit = num_re.match(r[-1]).groups()
        if unit.strip() == '':
            unit = None
        yield (indicator, name, value, unit)
    else:
        ids = indicator.split('\n')
        values = value.split('\n')
        for i in range(len(values)-1, -1, -1):
            v, u = num_re.match(values[i]).groups()
            if u.strip() == '':
                u = None
            try:
                yield (ids[i], name, v, u)
            except:
                yield (indicator, name, v, u)

    
#     return (indicator, name, value, unit)

In [272]:
to_df(tables_bs[0])

,0,1,2,3,4
0,Tepic,M ICS\nlndicator Number,M DG\nlndicator Number,lndicator,Value
1,Malaria,3.12,None,Household availability of insecticide-treated ...,34.2 percent
2,None,3.16,None,Malaria diagnostics usage,28.0 Percent
3,None,3.17,None,Antimalarial treatment of children under 5 the...,27.1 percent
4,None,3.18,6.8,Antimalarial treatment of children under age 5,51.2 percent
5,None,3.20,None,Intermittent preventive treatment for malaria,19.0 percent
6,WATER AND SANITATION,None,None,None,None
7,Water and,4.1,7.8,Use of improved drinking water sources,68.7 percent
8,sanitation,4.2,None,Water treatment,9.0 percent
9,None,4.3,7.9,Use of improved sanitation,7.4 percent


In [103]:
def extract_2012(t):
    
    res = []
    for i, row in t.iterrows():
        for a, b, c, d in process_row_2012(row):
            res.append((a,b,c,d))

    return pd.DataFrame(res, columns=['id', 'name', 'value', 'unit'])

In [129]:
res = []
for i, row in to_df(tables_bs[4]).iterrows():
    for a, b, c, d in process_row_2012(row):
        res.append((a,b,c,d))
        
pd.DataFrame(res, columns=['id', 'name', 'value', 'unit'])

,id,name,value,unit


In [420]:
res = extract_tbs_all(f, 0, 1, extract_2012)

0
1


In [421]:
(pd.concat(res, ignore_index=True)[['id', 'name', 'value', 'unit']]
 .to_csv(os.path.join(out_dir, f_.replace('.html', '.csv')), index=False))

In [ ]:
# 2015

In [835]:
def fill_names(df):
    for i, v in df.iterrows():
        if len(v.dropna()) == 1:  # only value line is here.
#             print(v.dropna())
            if not re.match('^[0-9\,\.\n]+$', v.dropna().values[0]):
                continue
            v.iloc[-1] = v.dropna().values[0]
#             print(v.iloc[:-1])
#             print(df.ix[i-1].values[:-1])
            try:
                v.iloc[:-1] = df.ix[i-1].values[:-1]
            except:
                print('error setting values, pass')
                v.iloc[:-1] = '-'
            df.ix[i] = v
        elif len(v.dropna()) > 1:
            if re.match('^[0-9\,\.\n]+$', v.dropna().values[-1]):  # if the row has numeric value in it, but no id in the first column
                matches = df.ix[:i, 0].str.match('[0-9\.\-]+')
                if len(matches[matches==True]) > 0:
                    last_idx = matches[matches==True].index[-1]
#                     df.ix[i, df.columns[-1]] = v.dropna().values[-1]
#                     for c in df.columns[:-1]:
#                         df.ix[i, c] = ' '.join(df.ix[last_idx:i, c].dropna().values)
                    df.ix[i, 0] = ' '.join(df.ix[last_idx:i, 0].dropna().values)
    return df

In [1110]:
to_df(tables_bs[0])

,0,1,2,3
0,Topic,MICS4 MDG\nIndicator Indicator\nNumber Number ...,Value,Unit
1,CHILD MORTALITY,None,None,None
2,Child mortality,1.1 4.1 Under-five mortality rate,102,per thousand
3,None,1.2 4.2 Infant mortality rate,74,per thousand
4,NUTRITION,None,None,None
5,Nutritional status,1.8 Underweight prevalence\n2.1a Moderate and ...,None,None
6,None,25.0,percent,None
7,None,10.6,percent,None
8,None,51.6,percent,None
9,None,34.1,percent,None


In [1017]:
def process_rows_2015(r, remove_col=None):  # each row of table
    num_re = re.compile('^[0-9\,\.\n]+$')
    id_re = re.compile('[0-9S\.\-]+')
    
    r = r.dropna().values
    
    if remove_col:
        if remove_col == -1:
            r = r[:-1]
        else:
            r = np.r_[r[:remove_col], r[remove_col+1:]]
    
    if len(r) < 2:
        return
    if not num_re.match(r[-1]):
        return
    
    if len(r) == 2:  # name and value
        indicator = r[0].split(' ')[0]
        name = ' '.join(r[0].split(' ')[1:])
        desc = None
        value = r[-1]
    elif len(r) == 3:
        indicator = r[0].split(' ')[0]
        name = ' '.join(r[0].split(' ')[1:])
        desc = r[1]
        value = r[-1]
    else:
        indicator = r[0]
        value = r[-1]
        desc = r[-2]
        name = r[-3]
        
    if not id_re.match(indicator):
        return

    if '\n' in indicator and '\n' in value:
        ids = indicator.split('\n')
        values = value.split('\n')
        if len(ids) == len(values):
            for i in range(len(ids)):
                yield (ids[i], name, desc, values[i])
        else:
            assert len(values) % len(ids) == 0
            for i in range(len(ids)):
                n = int(len(values) / len(ids))
                for p in range(n):
                    yield (ids[i], name, desc, values[i*n+p])
                
    elif '\n' in indicator:
        ids = indicator.split('\n')
        try:
            assert '\n' not in value
        except:
            print(indicator, values)
            raise
        for i in ids:
            yield (i ,name, desc, value)
    elif '\n' in value:
        values = value.split('\n')
        try:
            assert '\n' not in indicator
        except:
            print(values, indicator)
        for i in values:
            yield (indicator, name, desc, i)
    else:
        yield (indicator, name, desc, value)

In [1018]:
rows = []
for i, r in fill_names(to_df(tables_bs[13])).iterrows():
    for a, b, c, d in process_rows_2015(r, remove_col=-1):
        if b:
            b = b.strip()
        rows.append((a,b,c,d))

In [1019]:
pd.DataFrame(rows)

,0,1,2,3
0,7.1,Literacy rate among young women,Percentage of young women age 15-24 years who ...,99.1
1,7.2,School readiness,Percentage of children in first grade of prima...,98.1
2,7.3,Net intake rate in primary education,Percentage of children of school-entry age who...,97.0
3,7.4,Primary school net attendance ratio (adjusted),Percentage of children of primary school age c...,98.8
4,7.5,Secondary school net attendance ratio (adjusted),Percentage of children of secondary school age...,93.5
5,7.5 SS,Lower secondary school net attendance ratio (a...,Percentage of children of lower secondary scho...,96.2
6,7.5 SS SS,Upper secondary school net attendance ratio (a...,Percentage of children of upper secondary scho...,89.1
7,7.6,Children reaching last grade of primary,Percentage of children entering the first grad...,99.8
8,7.7,Primary completion rate,Number of children attending the last grade of...,92.4
9,7.8,Transition rate to secondary school,Number of children attending the last grade of...,99.6


In [1025]:
def extract_2015(t, remove_col=None):
    
    t = fill_names(t)
    
    if len(t.columns) < 2:
        print('not enough columns, skipping')
        return
                
    # do the extraction
    rows = []
    for i, r in t.iterrows():
        for a, b, c, d in process_rows_2015(r, remove_col=remove_col):
            rows.append((a, b, c, d))
            
    return pd.DataFrame(rows, columns=['id', 'name', 'description', 'value'])

In [1432]:
res = extract_tbs_all(f, 4, 31, extractor=extract_2015)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [1433]:
(pd.concat(res, ignore_index=True)[['id', 'name', 'description', 'value']]
 .to_csv(os.path.join(out_dir, f_.replace('.html', '.csv')), index=False))

In [1026]:
# for Serbia Roma Settlements and Serbia

ser_0 = extract_tbs_all(f, 5, 5, extractor=extract_2015)
ser_1 = extract_tbs_all(f, 6, 17, extractor=lambda x: (extract_2015(x, remove_col=-2)))

0
0
1
2
3
4
5
6
7
8
9
10
11


In [1027]:
ser = ser_0 + ser_1

ser_df = pd.concat(ser, ignore_index=True)

In [1029]:
ser_df.to_csv(os.path.join(out_dir, f_.replace('.html', '.Serbia Roma Settlements.csv')), index=False)

In [1030]:
srs = extract_tbs_all(f, 6, 17, extractor=lambda x: (extract_2015(x, remove_col=-1)))

0
1
2
3
4
5
6
7
8
9
10
11


In [1031]:
srs_df = pd.concat(srs, ignore_index=True)

In [1033]:
srs_df.to_csv(os.path.join(out_dir, f_.replace('.html', '.Serbia.csv')), index=False)

In [ ]:
id_re = re.compile('[0-9S\.]*')
num_re = re.compile('^\(?[0-9]+[\.|,]?[0-9]*\)?')

In [148]:
def extract_tbs(t):
    
    if len(t.columns) < 3:
        print('not enough columns, skipping')
        return
    
    if not np.any(t[0].str.match('^[1-9\-]+')):
        return
    
    # fill names
    if t[0].hasnans:
        for i, v in t.iterrows():
            names = list()
            if pd.isnull(v[0]):
                n = t[0].ix[:i].last_valid_index()
                if not n:
                    continue
                while not (re.match('^[1-9S]', t[0].ix[n]) or t[0].ix[n].startswith('-')):
                    names.append(t[0].ix[n])
                    n = n - 1
                names.append(t[0].ix[n])
                names.reverse()
                t.ix[n:i, 0] = ' '.join(names)
    
    ids = []
    values = []
    names = []

    for i, row in t.iterrows():
        r = row.dropna().astype(str).values
    #     print(r[0].strip())
        if num_re.match(str(r[-1]).strip()):
            if len(r) > 2:
                if 'MDG' in r[1]:
                    name = r[2]
                else:
                    name = r[1]
            else:
                name = np.nan
            if re.match('^[1-9S\-]', str(r[0])):
                ks_ = str(r[0]).split(' ')
                vs_ = str(r[-1]).split(' ')
                ks = []
                
                for k in ks_:
                    if k == '':
                        continue
                    k = k.strip()
                    if k.strip() == '-':
                        ks.append(k)
                    else:
                        if re.match("[0-9S\.]+[a-z]?$", k):
                            ks.append(k)
                        else:
                            name = ' '.join(ks_[ks_.index(k):])
                            break
                    
                vs = [x for x in vs_ if x is not '']
                
                if len(ks) > len(vs):
                    print('warning: length not match:')
                    print(ks)
                    print(vs)
                    while len(ks) > len(vs):
                        vs.append(vs[-1])
                    
                if len(vs) > len(ks):
                    print('warning: length not match:')
                    print(ks)
                    print(vs)
                    while len(vs) > len(ks):
                        ks.append(ks[-1])
                    
                [ids.append(x) for x in ks]
                [names.append(name) for _ in ks]
                [values.append(x) for x in vs]
            else:
                ids.append(str(r[0]))
                values.append(str(r[-1]))
                names.append(name)
        else:
#             print('not num: ', r[-1])
            pass
    assert len(ids) == len(values)
    return pd.DataFrame({'id': ids, 'name': names, 'value': values})

In [149]:
extract_tbs(pd.read_html(tables_bs[12].prettify(), thousands='')[0])

['3.3', '3.5', '3.6']
['83.4']


,id,name,value
0,3.1,Tuberculosis immunization coverage,97.6
1,3.2,Polio immunization coverage,83.1
2,3.3,Pentavalent immunization coverage,83.4
3,3.5,Pentavalent immunization coverage,83.4
4,3.6,Pentavalent immunization coverage,83.4
5,3.4,Measles immunization coverage,90.2
6,3.8,Full immunization coverage,77.5


In [165]:
dfs = []

for i in range(6, 43):
    try:
        dfs.append(extract_tbs(pd.read_html(tables_bs[i].prettify(), thousands='')[0]))
    except:
        print(i)
        raise

# dfs = [extract_tbs(pd.read_html(tables_bs[x].prettify())[0]) for x in list(range(7, 47))]

['2.1a']
['12,3', '3,3']
['2.17a']
['4,6', '6,8']
not enough columns, skipping
['7.1']
['83,9', '88,8']
['8.5']
['26,9', '0,4']
['8.6']
['16,1', '1,5']
['8.12']
['(2)', 'elle', 'néglige', 'les', 'enfants,', '(3)', 'elle', 'discute', 'ses', 'opinions,', '(4)']
['9.5']
['16,2', '10,9']
['9.6']
['18,0', '8,2']
['9.9']
['39,1', '34,4']
['9.10']
['13,7', '16,8']
['9.12']
['5,7', '31,6']
['9.13']
['42,1', '36,6']
['9.14']
['39,9', '58,7']
['9.15']
['47,1', '61,0']
not enough columns, skipping
['10.3']
['21,1', '43,0']
['11.2']
['86,4', '81,6']
['11.3']
['47,2', '44,3']


In [166]:
adf = pd.concat(dfs, ignore_index=True)

In [167]:
adf

,id,name,value
0,1.1,Taux de mortalité néonatale,21
1,1.2,OMD 4.2,36
2,1.3,Taux de mortalité post- néonatale,15
3,1.4,Taux de mortalité juvénile,17
4,1.5,OMD 4.1,52
5,2.1a,OMD 1.8 Prévalence de l’insuffisance Pourcent...,"12,3"
6,2.1a,OMD 1.8 Prévalence de l’insuffisance Pourcent...,"3,3"
7,2.2a,Prévalence du retard de croissance Modérée et...,"21,2"
8,2.2b,Prévalence du retard de croissance Modérée et...,"8,2"
9,2.3a,Prévalence de l’émaciation Modérée et Sévère ...,"8,2"


In [169]:
adf.to_csv('../output/stage2/Congo_2014_15_French.csv', index=False)

In [220]:
if re.match("^[0-9S\.]+", 'viii'):
    print(':P')

In [147]:
re.match("[1-9S\.]+[a-z]?$", '2.10')

In [128]:
a = pd.read_html(tables_bs[47].prettify(), thousands='')[0]

In [132]:
np.any(a[0].str.match('^[0-9S\-]'))

True

In [137]:
'12'.split(' ')

['12']

In [163]:
s1 = os.listdir('../source/to_proceed/')

In [164]:
s2 = os.listdir('../output/stage1/')

In [165]:
s1 = set([x[:-3] for x in s1 if 'pdf' in x])
s2 = set([x[:-4] for x in s2 if 'html' in x])

In [166]:
len(s2)

157

In [167]:
len(s1)

167

In [347]:
def to_df(tbl):
    tb = []

    for r in tbl.find_all('tr'):
        tr = []
        for td in r.find_all('td'):
            txts = []
            for c in td.children:
                if c.text.strip() != '':
                    txts.append(c.text.strip())
            txt = '\n'.join(txts)
            tr.append(txt)
        tb.append(tr)

In [346]:
pd.DataFrame(tb)[0].values

array([ 'MICS                 Indicator                     Description Indicator',
       '4.4 Safe disposal of child’s faeces Percentage of children age 0-2 years whose last stools\nwere disposed of safely',
       '4.5 Place for hand washing Percentage of households with a specific place for\nhand washing where water and soap or other cleansing agent are present',
       '4.6             Availability of soap or other             Percentage of households with soap or other cleansing cleansing agent agent',
       'REPRODUCTIVE HEALTH7', 'Early childbearing',
       '5.2 Early childbearing Percentage of ever-married women age 20-24 years\nwho had at least one live birth before age 18',
       'Maternal and new-born health',
       'MDG Antenatal care coverage Percentage of ever-married women age 15-49 years\n5.5                                                                  with a live birth in the last 5 years who were attended during their last pregnancy that led to a live birth\n

In [325]:
aaa = tables_bs[6]

In [333]:
aaa.extract?

In [1006]:
rrr = [1,2,3,4]

In [1008]:
rrr[:-2]

[1, 2]

In [1010]:
rrr[-2+1:]

[4]

In [1011]:
np.r_[rrr[:-2], rrr[-2+1:]]

array([1, 2, 4])

In [1228]:
id_re = re.compile('^[1-9]+|^TA|^SW')

In [1231]:
id_re.match('TA.1')

<_sre.SRE_Match object; span=(0, 2), match='TA'>

In [143]:
u'TA.1' == u'ТА.1'

False

In [145]:
ord('T')

84

In [146]:
ord('Т')

1058

In [164]:
'^MT'.encode('utf8') == '^МТ'.encode('utf-8')

False